In [ ]:
import {Tokenizer} from '/web/torus/neuro-module/tokenizer/tokenizer.js';
import {torus} from '/web/torus/torus.js';
//import {torus} from 'https://current.odant.org/web/torus/torus1.js';
import {nn} from '/web/torus/neuro-module/neuro-module.js';
import '/web/torus/neuro-module/loss-chart/loss-chart.js';
//import '/web/core/item/lib/structure-tree/structure-tree.js';

In [ ]:
DIM = 16;
char_step = 3; 
win_size = 10;
negative_size = 3;
intersection = true;
file_name = `tokenizer_model_${DIM}_${char_step}_${win_size}_${negative_size}_${intersection}.json`;
try{
    model = await this.loadFile(file_name);    
   // print('model loaded', model);
}
catch(e){
   model = null; 
  // print(e.message);
}
//model = null; 
>file_name

<label bold onclick='_findCodeEntry(this)' style='text-decoration: underline; padding: 2px; font-size: large; margin-bottom: 4px; cursor: pointer; color: -webkit-link'>file_name</label>

tokenizer_model_16_3_10_3_true.json

In [ ]:
if(model) {
    tensor = torus
    emb = new Tokenizer(model);
    >'токенайзер создан из модели'
}
else
    emb = new Tokenizer(DIM, char_step, win_size, negative_size, intersection);
>emb

<label bold onclick='_findCodeEntry(this)' style='text-decoration: underline; padding: 2px; font-size: large; margin-bottom: 4px; cursor: pointer; color: -webkit-link'>emb</label>

Tokenizer (dim: 16, char_step: 3, win_size: 10, negative_size: 3, intersection: true)


In [ ]:
host = this;
content = 'мама мыла раму';
ODA({is: 'oda-file-open',
    template: `
        <style>
            :host{
                @apply --vertical;
            }
        </style>
        <h2>
            Загрузка текстового файла
        </h2>
        <input accept=".txt" type="file" @change="readFile">
    `,
    content: {
        $def: true,
        $attr: true
    },
    readFile(e){
        let file = e.target.files[0];
        if (!file) 
            return;
        let reader = new FileReader();
        reader.onload = function(e) {
            host.saveFile(e.target.result, file.name)
        };
        reader.readAsText(file);
    }
})
>'oda-file-open'
tree = this.createElement('odant-structure-tree', {
    contextItem: this.contextItem, 
    allowFocus: true,
    content: true,
    doubleClickFocusMode: false,
    itemsList: '$$files'
});
tree.addEventListener('focused-item-changed', async (e)=>{
    if(e.detail.value){
        host.textFile = e.detail.value;
        content = await host.textFile.load();  
        console.dir(host.textFile)
        ODA.push('выбран файл: '+host.textFile?.label)
    }
}) 
>tree

<div style='cursor: pointer' onclick='_findCodeEntry(this)'>oda-file-open</div>

<label bold onclick='_findCodeEntry(this)' style='text-decoration: underline; padding: 2px; font-size: large; margin-bottom: 4px; cursor: pointer; color: -webkit-link'>tree</label>

In [ ]:
>content


<label bold onclick='_findCodeEntry(this)' style='text-decoration: underline; padding: 2px; font-size: large; margin-bottom: 4px; cursor: pointer; color: -webkit-link'>content</label>

мама мыла раму

In [ ]:
words = emb.tokenize(content, true);
>words

<label bold onclick='_findCodeEntry(this)' style='text-decoration: underline; padding: 2px; font-size: large; margin-bottom: 4px; cursor: pointer; color: -webkit-link'>words</label>

[vector " мама": shape(256), size(256), Float32Array, 
[[ -0.0572, -0.0350, -2.5e-3, -0.0466,  ...,      0.,      0.,      0.,      0.]]
,vector " мыла": shape(256), size(256), Float32Array, 
[[  0.0197, -8.9e-3, -0.0284,  0.0413,  ...,      0.,      0.,      0.,      0.]]
,vector " раму": shape(256), size(256), Float32Array, 
[[ -0.0147, -0.0143,  3.5e-3,  0.0493,  ...,      0.,      0.,      0.,      0.]]
]

In [ ]:
text = emb.print(words.map(w=>w.data));
>text

<label bold onclick='_findCodeEntry(this)' style='text-decoration: underline; padding: 2px; font-size: large; margin-bottom: 4px; cursor: pointer; color: -webkit-link'>text</label>

 мама мыла раму

In [ ]:
out = this.createElement('oda-tokenizer', {tokenizer: emb});
out.tokenizer = emb
chart = this.createElement('oda-loss-chart', {
    label: 'Chart',
    style: 'height: 300px',
    dataSet: []
});

progress = this.createElement('progress', {
    max: "100",
    value: "0",
    style: 'width: 100%'
});

emb.listen('progress', p =>{
    progress.value = p
})

chart.data = emb.losses

> out
> chart
> progress
cnt = 1
host = this;
btn = this.createElement('oda-button', {
    icon: 'icons:save', 
    successInvert: true, 
    iconSize: 64,
    label: `Save file "${file_name}"`,
    style: 'font-size: xx-large;',
    async click(e){
        btn.setAttribute('disabled', true);
        await host.saveFile(emb.model, file_name);  
        ODA.push('saved: '+ file_name);
    }
})

<label bold onclick='_findCodeEntry(this)' style='text-decoration: underline; padding: 2px; font-size: large; margin-bottom: 4px; cursor: pointer; color: -webkit-link'> out</label>

<label bold onclick='_findCodeEntry(this)' style='text-decoration: underline; padding: 2px; font-size: large; margin-bottom: 4px; cursor: pointer; color: -webkit-link'> chart</label>

<label bold onclick='_findCodeEntry(this)' style='text-decoration: underline; padding: 2px; font-size: large; margin-bottom: 4px; cursor: pointer; color: -webkit-link'> progress</label>

In [ ]:
torus.LEARNING_RATE = .3
// torus.USE_OUT_CACHE = true
torus.QUANTIZATION = 4

In [ ]:

cnt = 1

for(let i = 0; i<cnt; i++){
    await emb.train(content);
}
btn.removeAttribute('disabled');
cnt = 1
> btn
// print(emb.error);
// print(emb.tokens.map(i=>i.w))
>torus.LEARNING_RATE
> emb.losses.last

<label bold onclick='_findCodeEntry(this)' style='text-decoration: underline; padding: 2px; font-size: large; margin-bottom: 4px; cursor: pointer; color: -webkit-link'> btn</label>

<label bold onclick='_findCodeEntry(this)' style='text-decoration: underline; padding: 2px; font-size: large; margin-bottom: 4px; cursor: pointer; color: -webkit-link'>torus.LEARNING_RATE</label>

0.3

<label bold onclick='_findCodeEntry(this)' style='text-decoration: underline; padding: 2px; font-size: large; margin-bottom: 4px; cursor: pointer; color: -webkit-link'> emb.losses.last</label>

[0.1953230251012646]

In [ ]:
>emb.vocabulary


<label bold onclick='_findCodeEntry(this)' style='text-decoration: underline; padding: 2px; font-size: large; margin-bottom: 4px; cursor: pointer; color: -webkit-link'>emb.vocabulary</label>

{
  "0": {
    "w": "0",
    "t": "digits",
    "id": 250,
    "emb": {
      "$": "tensor",
      "shape": [
        16
      ],
      "isParam": false,
      "dType": "Float32Array",
      "data": "-0.53580 -0.85853 -0.68458 -0.69925 -0.06015 0.4135 0.3300 0.8263 0.1745 0.6456 -0.15297 -0.01730 -0.26080 0.2378 0.0592 0.2117"
    },
    "cnt": {
      "$": "tensor",
      "shape": [
        16
      ],
      "isParam": false,
      "dType": "Float32Array",
      "data": "0.4203 0.4431 0.5451 0.7145 -0.00265 0.2924 0.0894 -0.51785 0.3084 -0.38653 -0.42386 0.6747 0.2552 -0.20166 0.5173 -0.40562"
    },
    "count": 16,
    "next": {
      " ": 16
    },
    "error": 0.04142414405941963
  },
  "1": {
    "w": "1",
    "t": "digits",
    "id": 22,
    "emb": {
      "$": "tensor",
      "shape": [
        16
      ],
      "isParam": false,
      "dType": "Float32Array",
      "data": "-0.79723 -0.52684 -1.11706 -0.79890 0.4426 0.5769 1.4986 1.5082 0.8878 0.9731 -0.63427 -0.33480 -0.70433

In [ ]:


WordsTrain = class WordsTrain extends nn.Module{
    constructor(dim = 256, dim_hidden = 64, layer_count = 2, bias = false){
        super(...arguments);
    }
    __init__() {
        let step = Math.round((this.dim - this.dim_hidden) / this.layer_count);
        this.encoder = nn.Sequential(Array(this.layer_count).fill().map((_, i)=>{
            return nn.Linear(this.dim - i * step, this.dim - (i + 1) * step, this.bias);
        }));
        this.hidden = nn.Linear([this.dim_hidden], [this.dim_hidden], this.bias);
        this.decoder = nn.Sequential(Array(this.layer_count).fill().map((_, i)=>{
            return nn.Linear(this.dim_hidden + i * step, this.dim_hidden + (i + 1) * step, this.bias);
        }));
    }
    forward(word_token, train){
        let result = this.encoder.forward(word_token);
        console.log('encoder result: ', result);
        result = this.hidden.forward(result);
        console.log('hidden result: ', result);
        result = this.decoder.forward(result);
        console.log('decoder result: ', result);
        return result;
    }
}



In [ ]:
>words = emb.tokenize(content, true)
console.log(words)
net = new WordsTrain(256, 64, 2, true);
>net.toString()


<label bold onclick='_findCodeEntry(this)' style='text-decoration: underline; padding: 2px; font-size: large; margin-bottom: 4px; cursor: pointer; color: -webkit-link'>words = emb.tokenize(content, true)</label>

[vector мама: shape(256), size(256), Float32Array, 
[[  0.2143, -0.2113, -0.1446, -0.7481,  ...,      0.,      0.,      0.,      0.]]
,vector мыла: shape(256), size(256), Float32Array, 
[[ -0.1972, -0.3880, -0.8199, -0.3377,  ...,      0.,      0.,      0.,      0.]]
,vector раму: shape(256), size(256), Float32Array, 
[[ -0.6620, -0.9476, -0.2166, -0.3341,  ...,      0.,      0.,      0.,      0.]]
]

<label bold onclick='_findCodeEntry(this)' style='text-decoration: underline; padding: 2px; font-size: large; margin-bottom: 4px; cursor: pointer; color: -webkit-link'>net.toString()</label>

WordsTrain (dim: 256, dim_hidden: 64, layer_count: 2, bias: true, encoder: Sequential (Linear: Linear (bias: true, size_in: 160, size_out: 64)
  weight:
    PARAM:     matrix rand_init: Weights: shape(160,64), size(10 240), Float32Array, 
    [[  7.4e-5, -1.9e-5,  9.6e-5, -3.0e-5,  ..., -4.7e-5,  6.9e-6,  3.2e-5,  3.4e-7],
     [ -9.0e-5,  7.0e-5, -4.1e-5,  3.7e-5,  ..., -1.7e-5,  9.5e-5,  1.5e-6,  2.9e-5],
     [  1.5e-5,  7.8e-6,  8.6e-5,  7.3e-5,  ..., -7.9e-5,  8.2e-5,  6.3e-5, -3.7e-5],
     [  4.3e-5, -6.9e-5,  1.7e-5,  2.7e-5,  ...,  5.4e-7, -3.5e-5, -4.7e-5, -9.1e-6],
     [ -5.9e-5, -1.9e-5,  4.7e-6, -6.1e-5,  ..., -8.4e-6,  2.4e-5,  9.3e-5, -2.0e-5],
     [ -3.4e-5,  6.4e-5,  7.1e-6,  4.9e-5,  ...,  7.8e-5,  7.3e-5, -6.6e-5, -5.7e-5],
     [ -7.3e-5, -5.9e-5, -3.8e-5, -7.7e-5,  ..., -4.5e-5,  9.3e-5, -8.8e-6,  7.0e-5],
     [  3.1e-5, -6.2e-5, -2.9e-5,  4.8e-5,  ..., -2.4e-5, -6.6e-5, -2.0e-6, -3.1e-6],
     [  2.1e-5,  2.8e-5,  6.3e-6, -7.5e-5,  ...,  1.1e-5,  1.4e-5,  7.3e-

In [ ]:
for(let word of words){
    res = net(word)    
    >res
}


<label bold onclick='_findCodeEntry(this)' style='text-decoration: underline; padding: 2px; font-size: large; margin-bottom: 4px; cursor: pointer; color: -webkit-link'>res</label>

vector plus BIAS: shape(64), size(64), Float32Array, 
[[     NaN,     NaN,     NaN,     NaN,  ...,     NaN,     NaN,     NaN,     NaN]]


<label bold onclick='_findCodeEntry(this)' style='text-decoration: underline; padding: 2px; font-size: large; margin-bottom: 4px; cursor: pointer; color: -webkit-link'>res</label>

vector plus BIAS: shape(64), size(64), Float32Array, 
[[     NaN,     NaN,     NaN,     NaN,  ...,     NaN,     NaN,     NaN,     NaN]]


<label bold onclick='_findCodeEntry(this)' style='text-decoration: underline; padding: 2px; font-size: large; margin-bottom: 4px; cursor: pointer; color: -webkit-link'>res</label>

vector plus BIAS: shape(64), size(64), Float32Array, 
[[     NaN,     NaN,     NaN,     NaN,  ...,     NaN,     NaN,     NaN,     NaN]]
